In [8]:
import pandas as pd
from random import shuffle
from tqdm import tqdm
from sklearn.model_selection import train_test_split

## 数据预处理

In [10]:
corpus_data = pd.read_csv( "./data/corpus.tsv", sep="\t", names=["gid", "gname"])
print(f"商品标题corpus_data = {corpus_data.shape}")
train_data = pd.read_csv("./data/train.query.txt", sep="\t", names=["qid", "query"])
print(f"用户查询train_data = {train_data.shape}")
qrels = pd.read_csv("./data/qrels.train.tsv", sep="\t", names=["qid", "gid"])
print(f"查询与商品点击关系qrels = {qrels.shape}")
test_data = pd.read_csv("./data/dev.query.txt", sep="\t", names=["qid", "query"])
print(f"用户查询test_data = {test_data.shape}")

corpus_data = corpus_data.set_index("gid")
train_data = train_data.set_index("qid")
qrels = qrels.set_index("qid")
test_data = test_data.set_index("qid")

商品标题corpus_data = (1001500, 2)
用户查询train_data = (100000, 2)
查询与商品点击关系qrels = (100000, 2)
用户查询test_data = (1000, 2)


In [11]:
query2corpus = pd.DataFrame(data=corpus_data.loc[qrels.gid]['gname'].values,index=train_data.loc[qrels.index]['query'],columns=['gname'])

In [47]:
def get_count(prefix,L):
    prefix_l = len(str(prefix))
    if L%2==0:
        return 10**(L//2-prefix_l)
    else:
        return 10**(L//2+1-prefix_l)
get_count(11,3)

1

In [64]:
def kthPalindrome(queries, intLength: int):
    def get_count(prefix, L):
        prefix_l = len(str(prefix))
        if L % 2 == 0:
            return 10**(L//2-prefix_l)
        else:
            return 10**(L//2+1-prefix_l)

    def get_huiwen(prefix, L):
        if L%2 == 0:
            prefix = str(prefix)
            middle = ''
        else:
            prefix, middle = str(prefix)[:L//2],str(prefix)[L//2]
        s = f'{prefix}{middle}{prefix[::-1]}'
        return int(s)

    res = []
    L = intLength
    Max_prefix = 10**(L//2) if L %2 == 0 else 10**(L//2+1)
    for q in queries:
        prefix = 1
        if q>get_count(prefix,L)*10:
            res.append(-1)
            continue
        while q and prefix < Max_prefix:
            cnt = get_count(prefix, L)
            if q == 1 == cnt:
                break
            if q > cnt:
                prefix += 1
                q -= cnt
            else:
                prefix *= 10
        res.append(get_huiwen(prefix, L))
    return res
queries = [256965477,1231,1232,44,90,519833663,813131855,658305751,49,962390851,45]
intLength = 12
kthPalindrome(queries,intLength)

[-1,
 101230032101,
 101231132101,
 100043340001,
 100089980001,
 -1,
 -1,
 -1,
 100048840001,
 -1,
 100044440001]

In [68]:
input = [875,90,209,416,62,647,398,909,669,186,492,748,662,80,414,550,866,358,744,478,19,637,501,129,635,358,867,723,874,454,882,406,360,516,632,883,771,21,358,147,109,472,447,493903904,808,94,645,707,98043237,573,508,142,142,855,498,56,993,355,572,788,977,646,279,821,530,726,631,61,362,136,814,357,105,829909848,645,855,862,635,451,888,609788691,961592349,386,884,536,334,585,71,612]

In [65]:
ans = [100008747800001,100000898000001,100002080200001,100004151400001,100000616000001,100006464600001,100003979300001,100009080900001,100006686600001,100001858100001,100004919400001,100007474700001,100006616600001,100000797000001,100004131400001,100005494500001,100008656800001,100003575300001,100007434700001,100004777400001,100000181000001,100006363600001,100005000500001,100001282100001,100006343600001,100003575300001,100008666800001,100007222700001,100008737800001,100004535400001,100008818800001,100004050400001,100003595300001,100005151500001,100006313600001,100008828800001,100007707700001,100000202000001,100003575300001,100001464100001,100001080100001,100004717400001,100004464400001,-1,100008070800001,100000939000001,100006444600001,100007060700001,-1,100005727500001,100005070500001,100001414100001,100001414100001,100008545800001,100004979400001,100000555000001,100009929900001,100003545300001,100005717500001,100007878700001,100009767900001,100006454600001,100002787200001,100008202800001,100005292500001,100007252700001,100006303600001,100000606000001,100003616300001,100001353100001,100008131800001,100003565300001,100001040100001,-1,100006444600001,100008545800001,100008616800001,100006343600001,100004505400001,100008878800001,-1,-1,100003858300001,100008838800001,100005353500001,100003333300001,100005848500001,100000707000001,100006111600001]

In [66]:
mine = [100008747800001,100000898000001,100002080200001,100004151400001,100000616000001,100006464600001,100003979300001,100009080900001,100006686600001,100001858100001,100004919400001,100007474700001,100006616600001,100000797000001,100004131400001,100005494500001,100008656800001,100003575300001,100007434700001,100004777400001,100000181000001,100006363600001,100005000500001,100001282100001,100006343600001,100003575300001,100008666800001,100007222700001,100008737800001,100004535400001,100008818800001,100004050400001,100003595300001,100005151500001,100006313600001,100008828800001,100007707700001,100000202000001,100003575300001,100001464100001,100001080100001,100004717400001,100004464400001,-1,100008070800001,100000939000001,100006444600001,100007060700001,180432363234081,100005727500001,100005070500001,100001414100001,100001414100001,100008545800001,100004979400001,100000555000001,100009929900001,100003545300001,100005717500001,100007878700001,100009767900001,100006454600001,100002787200001,100008202800001,100005292500001,100007252700001,100006303600001,100000606000001,100003616300001,100001353100001,100008131800001,100003565300001,100001040100001,-1,100006444600001,100008545800001,100008616800001,100006343600001,100004505400001,100008878800001,-1,-1,100003858300001,100008838800001,100005353500001,100003333300001,100005848500001,100000707000001,100006111600001]

In [70]:
new_mine = [100008747800001,100000898000001,100002080200001,100004151400001,100000616000001,100006464600001,100003979300001,100009080900001,100006686600001,100001858100001,100004919400001,100007474700001,100006616600001,100000797000001,100004131400001,100005494500001,100008656800001,100003575300001,100007434700001,100004777400001,100000181000001,100006363600001,100005000500001,100001282100001,100006343600001,100003575300001,100008666800001,100007222700001,100008737800001,100004535400001,100008818800001,100004050400001,100003595300001,100005151500001,100006313600001,100008828800001,100007707700001,100000202000001,100003575300001,100001464100001,100001080100001,100004717400001,100004464400001,-1,100008070800001,100000939000001,100006444600001,100007060700001,-1,100005727500001,100005070500001,100001414100001,100001414100001,100008545800001,100004979400001,100000555000001,100009929900001,100003545300001,100005717500001,100007878700001,100009767900001,100006454600001,100002787200001,100008202800001,100005292500001,100007252700001,100006303600001,100000606000001,100003616300001,100001353100001,100008131800001,100003565300001,100001040100001,-1,100006444600001,100008545800001,100008616800001,100006343600001,100004505400001,100008878800001,-1,-1,100003858300001,100008838800001,100005353500001,100003333300001,100005848500001,100000707000001,100006111600001]

In [71]:
for i,m,a in zip(input,new_mine,ans):
    if a!=m:
        print(i,m,a)

37

In [ ]:
X_train,X_val = train_test_split(query2corpus,test_size=0.2)

In [ ]:
X_train.to_csv("./data/X_train.csv")

In [ ]:
import torch
from scipy.spatial.distance import cosine
from transformers import AutoModel, AutoTokenizer

# Import our models. The package will take care of downloading the models automatically
tokenizer = AutoTokenizer.from_pretrained("princeton-nlp/sup-simcse-bert-base-uncased")
model = AutoModel.from_pretrained("princeton-nlp/sup-simcse-bert-base-uncased")

# Tokenize input texts
texts = [
    "There's a kid on a skateboard.",
    "A kid is skateboarding.",
    "A kid is inside the house."
]
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

# Get the embeddings
with torch.no_grad():
    embeddings = model(**inputs, output_hidden_states=True, return_dict=True).pooler_output

# Calculate cosine similarities
# Cosine similarities are in [-1, 1]. Higher means more similar
cosine_sim_0_1 = 1 - cosine(embeddings[0], embeddings[1])
cosine_sim_0_2 = 1 - cosine(embeddings[0], embeddings[2])

print("Cosine similarity between \"%s\" and \"%s\" is: %.3f" % (texts[0], texts[1], cosine_sim_0_1))
print("Cosine similarity between \"%s\" and \"%s\" is: %.3f" % (texts[0], texts[2], cosine_sim_0_2))
inputs

Cosine similarity between "There's a kid on a skateboard." and "A kid is skateboarding." is: 0.943
Cosine similarity between "There's a kid on a skateboard." and "A kid is inside the house." is: 0.439


{'input_ids': tensor([[  101,  2045,  1005,  1055,  1037,  4845,  2006,  1037, 17260,  6277,
          1012,   102],
        [  101,  1037,  4845,  2003, 17260, 21172,  1012,   102,     0,     0,
             0,     0],
        [  101,  1037,  4845,  2003,  2503,  1996,  2160,  1012,   102,     0,
             0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]])}